In [47]:
import pandas as pd
PATH = "C:/Users/jackl/OneDrive/Documents/finance_research/japan_qe/"

In [48]:
# textfile = open(PATH+"nikkei_sedols.txt", "w")
# for element in sorted(list(set(weight_df[weight_df['indexlistmnem']=='LJAPDOWA']['sedol']))):
#     textfile.write(element + "\n")
# textfile.close()

In [49]:
ticker_df = pd.read_csv(PATH+'raw_data/worldscope/ticker_sedol.csv')
ticker_df.columns = ['code', 'year_', 'freq', 'ticker', 'sedol']

In [50]:
weight_df = pd.read_pickle(PATH+'checkpoint_data/index_weights.pkl')
weight_df.columns = ['date_', 'weight_within_index', 'constintcode', 'indexlistmnem', 'month_yr_index',
                     'merged_date', 'sedol', 'month', 'yr', 'conm']
topix_df = weight_df[weight_df['indexlistmnem']=='LTOKYOSE'].sort_values('date_').drop_duplicates(['sedol'], keep='last')
nikkei_df = weight_df[weight_df['indexlistmnem']=='LJAPDOWA'].sort_values('date_').drop_duplicates(['sedol'], keep='last')

In [51]:
always_in = weight_df[(weight_df['date_'] > pd.to_datetime('2010-12-01')) & 
                      (weight_df['date_'] < pd.to_datetime('2021-01-01'))]

always_in = always_in[always_in['indexlistmnem']=='LJAPDOWA'].groupby(['sedol']).count()[['date_']].reset_index()
always_in = always_in[always_in['date_'] >= 121]
always_in['date_'] = 1
always_in.columns = ['sedol', 'always_in']

In [52]:
nk_df = pd.merge(nikkei_df[['sedol', 'conm']], 
                 ticker_df[['ticker', 'sedol']].drop_duplicates(), 
                 on=['sedol'], 
                 how='left').sort_values(['ticker'])

In [53]:
nk_df = pd.merge(nk_df, 
                 always_in, 
                 on=['sedol'], 
                 how='left')

In [54]:
nk_df.to_csv(PATH+'nk_df_v2.csv', index=False)